#### Librerías, variables, funciones y lectura de datos.

In [ ]:
#1. Importo librerías.
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#2. Lectura de datos.
series_temporales = pd.read_csv("./inputs/dataset.csv")
diccionario = pd.read_excel("./inputs/diccionario.xlsx",sheet_name="metadata")

#### EDA.

In [ ]:
#1. Columnas y variables.
print("Tenemos {} columnas. Estas son: {}".format(series_temporales.shape[1],series_temporales.columns))

In [ ]:
#2. Información general de las variables.
series_temporales.info() 

# Tenemos series temporales con datos nulos......

In [ ]:
#3. Convertimos tipo de datos de la fecha.
series_temporales["fecha"] = pd.to_datetime(series_temporales["fecha"])

In [ ]:
#4. Borramos filas enteras con NA.
series_temporales.dropna(inplace=True)

In [ ]:
#5. Entendemos las fechas de inicio y fin de la Serie temporal
inicio = series_temporales["fecha"].sort_values(ascending=True).iloc[0]
fin = series_temporales["fecha"].sort_values(ascending=False).iloc[0]

print(f"Las series temporales son algunas diarias, y otras mensuales, yendo desde {inicio} a {fin}.")

In [ ]:
#6. Tiene fechas repetidas?
print("Fechas unicas: {}".format(series_temporales["fecha"].nunique()))
print("Cantidad de filas totales: {}".format(series_temporales.shape[0]))

In [ ]:
#7. Entre el inicio y fin hay 170 fechas?
print("Tenemos una apertura de fechas de {} días".format(fin - inicio))
print("Debería cubrir {} días".format(series_temporales.shape[0]))

In [ ]:
#8. Que fechas faltan?
inicio = series_temporales["fecha"].min()
fin = series_temporales["fecha"].max()

rango_completo = pd.date_range(start=inicio, end=fin, freq='D')

fechas_reales = series_temporales["fecha"].unique()

fechas_faltantes = rango_completo.difference(fechas_reales)

print("Cantidad de fechas faltantes:", len(fechas_faltantes))
print("Fechas faltantes:")
print(fechas_faltantes)

# Tenemos fechas faltantes... las series temporales para considerarse tales tienen que ser constantes en sus intervalos.


In [ ]:
#9. Analisis estadísticos descriptivos básicos de las series temporales.
series_temporales.describe()

#### Análisis básicos en el Dominio del Tiempo.

In [ ]:
# Aseguramos que la columna fecha es datetime y está ordenada
series_temporales["fecha"] = pd.to_datetime(series_temporales["fecha"])
series_temporales = series_temporales.sort_values("fecha")

# Listamos las columnas numéricas (excluyendo 'fecha')
variables = series_temporales.drop(columns=["fecha"]).columns

# Graficamos cada variable en un gráfico separado
for var in variables:
    plt.figure(figsize=(12,5))  # Nueva figura para cada variable
    plt.plot(series_temporales["fecha"], series_temporales[var], marker='.')
    plt.title(f"Evolución de {var} en el tiempo")
    plt.xlabel("Fecha")
    plt.ylabel(var)
    plt.grid(True)
    plt.show()


#### Análisis en el Dominio de la Frecuencia (con TTF).

In [ ]:
# -------------------------
# 0️⃣ Configuración inicial
# -------------------------
# Supongamos que:
# series_temporales -> DataFrame con todas las series, columna 'fecha'
# diccionario -> DataFrame con columnas ['serie', 'frecuencia'] indicando 'D' o 'M'

# Aseguramos que fecha sea datetime
series_temporales["fecha"] = pd.to_datetime(series_temporales["fecha"])

# Ordenamos por fecha
series_temporales = series_temporales.sort_values("fecha")

# -------------------------
# 1️⃣ Iterar sobre cada variable
# -------------------------
for _, row in diccionario.iterrows():
    var = row['serie']
    freq = row['frecuencia']  # 'D' o 'M'
    
    # Extraer la serie
    y = series_temporales[['fecha', var]].copy()
    y = y.set_index('fecha')
    
    # Reindexar según frecuencia real
    if freq == 'diaria':
        y = y.asfreq('D')
        d = 1   # 1 día entre muestras
    elif freq == 'mensual':
        y = y.asfreq('M')
        d = 30  # aprox 30 días entre muestras
    
    # Rellenar NaN con interpolación lineal
    y_filled = y.interpolate(method='linear')
    
    # -------------------------
    # 2️⃣ Graficar serie temporal
    # -------------------------
    plt.figure(figsize=(12,5))
    plt.plot(y_filled.index, y_filled[var], marker='.')
    plt.title(f"Evolución de {var} ({freq})")
    plt.xlabel("Fecha")
    plt.ylabel(var)
    plt.grid(True)
    plt.show()
    
    # -------------------------
    # 3️⃣ Graficar FFT global
    # -------------------------
    N = len(y_filled)
    yf = np.fft.fft(y_filled[var].values)
    xf = np.fft.fftfreq(N, d=d)[:N//2]  # frecuencias positivas
    amplitud = 2.0/N * np.abs(yf[:N//2])
    
    # Convertir a periodos en días
    periodo = 1/xf[1:]  # omitimos frecuencia cero
    amplitud_periodo = amplitud[1:]
    
    plt.figure(figsize=(12,5))
    plt.plot(periodo, amplitud_periodo, marker='.')
    plt.title(f"FFT global de {var} ({freq}) - Periodos en días")
    plt.xlabel("Periodo (días por ciclo)")
    plt.ylabel("Amplitud")
    plt.grid(True)
    plt.show()